<a href="https://colab.research.google.com/github/ngc-minh/ChatGPT-API/blob/master/notebooks/01_Adapter_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1️⃣ Training an Adapter for a Transformer Model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using the _[Adapters](https://github.com/Adapter-Hub/adapters)_ library and Hugging Face's _Transformers_ library.

We train a **[bottleneck adapter](https://docs.adapterhub.ml/methods.html#bottleneck-adapters)** on top of a pre-trained model here. Most of the code is identical to a full fine-tuning setup using _Transformers_.

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via Hugging Face's [_Datasets_](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -qq adapters datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.5 MB/s eta 0:00:00


## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [2]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

{'train': 8530, 'validation': 1066, 'test': 1066}

Every dataset sample has an input text and a binary label:

In [3]:
dataset['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [4]:
dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

In [5]:
dataset['train'][2]

{'text': 'effective but too-tepid biopic', 'label': 1}

Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [6]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
dataset

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})

In [8]:
dataset['train']

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 8530
})

In [9]:
dataset['train']['text']

Column(['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .', 'effective but too-tepid biopic', 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .', "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one ."])

In [10]:
dataset['train']['input_ids']

Column([[0, 627, 3152, 16, 19083, 7, 28, 5, 733, 620, 3220, 18, 92, 22, 2764, 260, 22, 8, 14, 37, 18, 164, 7, 146, 10, 17158, 190, 2388, 87, 4709, 282, 279, 8447, 5557, 329, 47129, 2156, 1236, 12001, 12, 3998, 5247, 3538, 9656, 1794, 50, 11235, 2987, 842, 9487, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 627, 39966, 9412, 10986, 18719, 9, 22, 5, 30722, 9, 5, 11987, 22, 26105, 16, 98, 1307, 14, 10, 6730, 9, 1617, 1395, 17327, 6190, 1029, 12, 9408, 73, 19709, 181, 5906, 10267, 1478, 18, 4939, 3360, 9, 1236, 479, 910, 479, 910, 479, 7, 462, 330, 2495, 18, 1692, 12, 25581, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 20278, 53, 350, 12, 90, 2462, 808, 4003, 18137, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1594, 47, 2128, 101,

In [11]:
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})

In [17]:
dataset['train']


Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 8530
})

Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model checkpoint from the Hugging Face Hub. We load it with [`AutoAdapterModel`](https://docs.adapterhub.ml/classes/models/auto.html), a class unique to `adapters`. In addition to regular _Transformers_ classes, this class comes with all sorts of adapter-specific functionality, allowing flexible management and configuration of multiple adapters and prediction heads. [Learn more](https://docs.adapterhub.ml/prediction_heads.html#adaptermodel-classes).

In [21]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and an adapter configuration. `"seq_bn"` denotes a [sequential bottleneck adapter](https://docs.adapterhub.ml/methods.html#bottleneck-adapters) configuration.
_Adapters_ supports a diverse range of different adapter configurations. For example, `config="lora"` can be passed for training a [LoRA](https://docs.adapterhub.ml/methods.html#lora) adapter, `config="prefix_tuning"` for [prefix tuning](https://docs.adapterhub.ml/methods.html#prefix-tuning) or `config="loreft"` for [LoReFT](https://docs.adapterhub.ml/methods.html#reft). You can find all currently supported configs [here](https://docs.adapterhub.ml/methods.html).

Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model, so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [ ]:
# Add a new adapter
model.add_adapter("rotten_tomatoes", config="seq_bn")
# Alternatively, e.g.:
# model.add_adapter("rotten_tomatoes", config="lora")

# Add a matching classification head
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("rotten_tomatoes")

For training an adapter, we make use of the `AdapterTrainer` class built-in into _Adapters_. This class is largely identical to _Transformer_'s `Trainer`, with some helpful tweaks e.g. for checkpointing only adapter weights.

We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full fine-tuning.** Adapter training usually requires a few more training epochs than full fine-tuning.

In [ ]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [ ]:
trainer.train()

  0%|          | 0/1602 [00:00<?, ?it/s]

{'loss': 0.5131, 'learning_rate': 8.75156054931336e-05, 'epoch': 0.75}
{'loss': 0.3153, 'learning_rate': 7.503121098626716e-05, 'epoch': 1.5}
{'loss': 0.2828, 'learning_rate': 6.254681647940075e-05, 'epoch': 2.25}
{'loss': 0.271, 'learning_rate': 5.006242197253434e-05, 'epoch': 3.0}
{'loss': 0.25, 'learning_rate': 3.7578027465667915e-05, 'epoch': 3.75}
{'loss': 0.2428, 'learning_rate': 2.50936329588015e-05, 'epoch': 4.49}
{'loss': 0.2469, 'learning_rate': 1.2609238451935081e-05, 'epoch': 5.24}
{'loss': 0.2216, 'learning_rate': 1.2484394506866418e-07, 'epoch': 5.99}
{'train_runtime': 305.6866, 'train_samples_per_second': 167.426, 'train_steps_per_second': 5.241, 'train_loss': 0.2928388835525096, 'epoch': 6.0}


TrainOutput(global_step=1602, training_loss=0.2928388835525096, metrics={'train_runtime': 305.6866, 'train_samples_per_second': 167.426, 'train_steps_per_second': 5.241, 'train_loss': 0.2928388835525096, 'epoch': 6.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [ ]:
trainer.evaluate()

  0%|          | 0/34 [00:00<?, ?it/s]

{'eval_loss': 0.2768324911594391,
 'eval_acc': 0.8902439024390244,
 'eval_runtime': 3.1694,
 'eval_samples_per_second': 336.339,
 'eval_steps_per_second': 10.728,
 'epoch': 6.0}

We can put our trained model into a _Transformers_ pipeline to be able to make new predictions conveniently:

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

The model 'RobertaAdapterModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 

[{'label': '👍', 'score': 0.98734450340271}]

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [ ]:
model.save_adapter("./final_adapter", "rotten_tomatoes")

!ls -lh final_adapter

Der Befehl "ls" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


**Share your work!**

The final step after successful training is to share our adapter with the world!
_Adapters_ seamlessly integrates with the [Hugging Face Model Hub](https://huggingface.co/models), so you can publish your trained adapter with a single method call:

**Important:** Make sure you're properly authenticated with your Hugging Face account before running this method. You can log in by running `huggingface-cli login` on your terminal.

In [ ]:
model.push_adapter_to_hub(
    "my-awesome-adapter",
    "rotten_tomatoes",
    datasets_tag="rotten_tomatoes"
)

This will create a repository _my-awesome-adapter_ under your username, generate a default adapter card as README.md and upload the adapter named `rotten_tomatoes` together with the adapter card to the new repository. [Learn more](https://docs.adapterhub.ml/huggingface_hub.html).

➡️ Continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapters/blob/main/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.